# Prompt Chaining Design Pattern: Order Enquiry Agent

## Overview
This notebook demonstrates the **Prompt Chaining** pattern - a workflow where the output of one LLM call becomes the input to the next in a predetermined sequence. Unlike agents that dynamically decide their actions, prompt chains follow a fixed path with conditional branching.

## Use Case
An automated customer service agent that:
1. Extracts order information from natural language queries
2. Looks up order details in a database
3. **Validates** the lookup result and handles errors
4. Retrieves current shipping status (if order found)
5. Generates a personalized response

## Architecture
```
User Query → Extract Info → Database Lookup → Validation
                                                  ├─ Not Found → END (with error message)
                                                  └─ Found → Check Status → Generate Response → END
```

### Key Pattern: Validation Node
The **validation node** acts as a gateway that stops the flow when an order is not found, avoiding unnecessary processing and providing immediate error feedback.

## 1. Setup & Dependencies

Install required packages for building the prompt chaining workflow with LangGraph.

In [ ]:
!pip install -q langgraph langchain-core langchain-openai python-dotenv

Import necessary libraries for state management, message handling, and LLM interactions.

In [ ]:
import os
import json
from typing import Dict, Optional, List, TypedDict
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# LangGraph imports
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

# LangChain OpenAI
from langchain_openai import ChatOpenAI

print("✅ Dependencies loaded successfully")

## 2. Environment Configuration

Configure OpenAI API credentials. For Google Colab, use userdata to securely store your API key.

In [ ]:
# For Google Colab - Load API key from userdata
try:
    from google.colab import userdata
    os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
    print("✅ API key loaded from Google Colab userdata")
except ImportError:
    # For local environment - set your API key directly
    os.environ["OPENAI_API_KEY"] = "your-api-key-here"
    print("⚠️  Running locally - Please set your API key")

# Initialize ChatOpenAI model
llm = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0.2,
    api_key=os.environ.get("OPENAI_API_KEY")
)

print("✅ ChatOpenAI model configured")
print(f"   Model: {llm.model_name}")
print(f"   Temperature: {llm.temperature}")

## 3. Load Data from JSON Files

Load customer and order data from JSON files. The data includes complete information with shipping details, tracking numbers, and order history.

In [ ]:
# Load data from JSON files
def load_data():
    """Load customers and orders from JSON files"""
    data_dir = 'data'
    
    # Load customers
    with open(f'{data_dir}/customers.json', 'r') as f:
        customers_list = json.load(f)
    
    # Load orders
    with open(f'{data_dir}/orders.json', 'r') as f:
        orders_list = json.load(f)
    
    # Convert to dictionaries for fast lookup
    customers_dict = {c['customer_id']: c for c in customers_list}
    orders_dict = {o['order_id']: o for o in orders_list}
    
    return customers_dict, orders_dict

# Load the data
CUSTOMERS, ORDERS = load_data()

print("✅ Data loaded successfully from JSON files")
print(f"📊 Customers: {len(CUSTOMERS)}, Orders: {len(ORDERS)}")
print(f"\n📋 Sample Customer IDs: {list(CUSTOMERS.keys())[:5]}")
print(f"📦 Sample Order IDs: {list(ORDERS.keys())[:5]}")

## 4. State Definition

Define the state schema that will be passed through each step of the prompt chain. Each node will read from and update this shared state.

In [ ]:
class OrderEnquiryState(TypedDict):
    """State that flows through the prompt chain"""
    user_query: str                          # Original user question
    extracted_info: Optional[Dict]           # Step 1: Extracted order/customer info
    order_data: Optional[Dict]               # Step 2: Retrieved order data
    customer_data: Optional[Dict]            # Step 2: Retrieved customer data
    order_found: bool                        # Routing decision
    status_info: Optional[Dict]              # Step 3: Status and shipping info
    final_response: str                      # Step 4: Generated response
    
print("✅ State schema defined")

## 5. Prompt Chain Step 1: Extract Information

This node uses an LLM to extract structured information (order ID, customer details) from natural language queries.

In [ ]:
def extract_info_node(state: OrderEnquiryState) -> Dict:
    """
    Step 1: Extract order ID and customer information from user query
    """
    print("\n🔍 STEP 1: Extracting order information...")
    
    user_query = state["user_query"]
    
    system_prompt = """You are an information extraction assistant. 
Extract order ID and customer information from user queries.

Return ONLY a JSON object with this format:
{
  "order_id": "order number or null",
  "customer_name": "customer name or null",
  "customer_id": "customer ID or null"
}

Examples:
- "Where is my order 101?" → {"order_id": "101", "customer_name": null, "customer_id": null}
- "I'm Sarah, checking order 101" → {"order_id": "101", "customer_name": "Sarah", "customer_id": null}
"""
    
    try:
        # Use LangChain ChatOpenAI with messages
        messages = [
            SystemMessage(content=system_prompt),
            HumanMessage(content=user_query)
        ]
        
        # Invoke the model with temperature=0 for extraction
        extraction_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
        response = extraction_llm.invoke(messages)
        
        extracted = json.loads(response.content)
        print(f"   Extracted: {json.dumps(extracted, indent=2)}")
        
        return {"extracted_info": extracted}
        
    except Exception as e:
        print(f"   ❌ Error: {e}")
        return {"extracted_info": {"order_id": None, "customer_name": None, "customer_id": None}}

print("✅ Extract info node defined")

## 6. Prompt Chain Step 2: Database Lookup

Query the database with extracted information to retrieve order and customer details. This step determines the routing path.

In [ ]:
def database_lookup_node(state: OrderEnquiryState) -> Dict:
    """
    Step 2: Look up order in database
    """
    print("\n💾 STEP 2: Querying database...")
    
    extracted = state.get("extracted_info", {})
    order_id = extracted.get("order_id")
    
    if not order_id:
        print("   ❌ No order ID found in query")
        return {
            "order_data": None,
            "customer_data": None,
            "order_found": False
        }
    
    # Look up order
    order_data = ORDERS.get(order_id)
    
    if not order_data:
        print(f"   ❌ Order {order_id} not found")
        return {
            "order_data": None,
            "customer_data": None,
            "order_found": False
        }
    
    # Look up customer
    customer_data = CUSTOMERS.get(order_data["customer_id"])
    
    print(f"   ✅ Order {order_id} found for {customer_data['name']}")
    print(f"   Status: {order_data['status']}")
    
    return {
        "order_data": order_data,
        "customer_data": customer_data,
        "order_found": True
    }

print("✅ Database lookup node defined")

## 7. Prompt Chain Step 3: Validation Node

This node validates whether the order was found after database lookup. If not found, it generates an error response and stops the flow. If found, the flow continues to check status.

In [ ]:
def validation_node(state: OrderEnquiryState) -> Dict:
    """
    Step 3: Validate if order was found
    If not found, generate error response and set flag to stop flow
    """
    print("\n✅ STEP 3: Validating order lookup...")
    
    order_found = state.get("order_found", False)
    
    if not order_found:
        # Generate error response when order not found
        print("   ⚠️  Order not found - Generating error response")
        
        extracted = state.get("extracted_info", {})
        order_id = extracted.get("order_id", "unknown")
        
        system_prompt = """You are a helpful customer service representative.
Generate a polite error message for an order that wasn't found.

Guidelines:
- Be empathetic and apologetic
- Suggest possible reasons (typo, wrong number, etc.)
- Provide clear next steps
- Include customer support contact: 1-800-SHOP-HELP
- Keep it brief and friendly
"""
        
        user_prompt = f"Order ID '{order_id}' was not found in our system. Generate an appropriate response."
        
        try:
            # Use LangChain ChatOpenAI
            messages = [
                SystemMessage(content=system_prompt),
                HumanMessage(content=user_prompt)
            ]
            
            response = llm.invoke(messages)
            final_response = response.content
            
            print(f"   ✅ Error response generated - Flow will stop")
            
            return {"final_response": final_response}
            
        except Exception as e:
            print(f"   ❌ Error generating response: {e}")
            return {
                "final_response": f"I couldn't find order {order_id}. Please contact support at 1-800-SHOP-HELP."
            }
    else:
        # Order found - continue to next step
        print("   ✅ Order found - Continuing to status check")
        return {}

print("✅ Validation node defined")

## 8. Prompt Chain Step 4: Check Status

This node retrieves the current order status and enriches it with tracking information and delivery details.

In [ ]:
def check_status_node(state: OrderEnquiryState) -> Dict:
    """
    Step 4: Retrieve and format current order status
    """
    print("\n📦 STEP 4: Checking order status...")
    
    order_data = state["order_data"]
    customer_data = state["customer_data"]
    
    # Prepare status information
    status_info = {
        "order_id": order_data["order_id"],
        "customer_name": customer_data["name"],
        "status": order_data["status"],
        "items": order_data["items"],
        "total_amount": order_data["total_amount"],
        "order_date": order_data["order_date"],
        "estimated_delivery": order_data["estimated_delivery"],
        "shipping_address": order_data["shipping_address"],
        "tracking_number": order_data.get("tracking_number"),
        "carrier": order_data["carrier"],
        "last_update": order_data["last_update"],
        "actual_delivery": order_data.get("actual_delivery")
    }
    
    print(f"   ✅ Status retrieved: {status_info['status']}")
    print(f"   Latest update: {status_info['last_update']}")
    
    return {"status_info": status_info}

print("✅ Check status node defined")

## 9. Prompt Chain Step 5: Generate Success Response

This is the final step that takes all gathered information and generates a friendly, conversational response for the customer.

In [ ]:
def generate_success_response_node(state: OrderEnquiryState) -> Dict:
    """
    Step 5: Generate personalized success response
    """
    print("\n✨ STEP 5: Generating personalized response...")
    
    status_info = state["status_info"]
    
    system_prompt = """You are a friendly customer service representative. 
Generate a warm, personalized response about the customer's order.

Guidelines:
- Address customer by name
- Clearly state order status
- Include delivery information
- Add tracking details if available
- Be conversational and helpful
- Keep it concise (3-4 sentences)
"""
    
    user_prompt = f"""Create a response for this order:

{json.dumps(status_info, indent=2)}
"""
    
    try:
        # Use LangChain ChatOpenAI
        messages = [
            SystemMessage(content=system_prompt),
            HumanMessage(content=user_prompt)
        ]
        
        response = llm.invoke(messages)
        final_response = response.content
        
        print(f"   ✅ Response generated ({len(final_response)} chars)")
        
        return {"final_response": final_response}
        
    except Exception as e:
        print(f"   ❌ Error: {e}")
        return {"final_response": f"Your order #{status_info['order_id']} is {status_info['status']}."}

print("✅ Generate success response node defined")

## 10. Routing Logic After Validation

Define conditional routing after validation. If order not found (error response already generated), go to END. Otherwise, continue to check status.

In [ ]:
def route_after_validation(state: OrderEnquiryState) -> str:
    """
    Route after validation node
    - If order found: continue to check_status
    - If order not found: go directly to END (error response already generated)
    """
    if state.get("order_found", False):
        print("\n🔀 ROUTING: Validation passed → Check status")
        return "check_status"
    else:
        print("\n🔀 ROUTING: Validation failed → END (error response set)")
        return "END"

print("✅ Routing function defined")

## 11. Build the Prompt Chain Graph

Construct the LangGraph workflow with a validation node that stops the flow when order is not found.

In [ ]:
def create_prompt_chain_graph():
    """Build the prompt chaining workflow with validation node"""
    
    # Initialize graph builder
    builder = StateGraph(OrderEnquiryState)
    
    # Add all nodes
    builder.add_node("extract_info", extract_info_node)
    builder.add_node("database_lookup", database_lookup_node)
    builder.add_node("validation", validation_node)  # New validation node
    builder.add_node("check_status", check_status_node)
    builder.add_node("generate_success", generate_success_response_node)
    
    # Define linear chain
    builder.add_edge(START, "extract_info")
    builder.add_edge("extract_info", "database_lookup")
    builder.add_edge("database_lookup", "validation")
    
    # Conditional routing after validation
    builder.add_conditional_edges(
        "validation",
        route_after_validation,
        {
            "check_status": "check_status",
            "END": END  # Direct to END when validation fails
        }
    )
    
    # Success path
    builder.add_edge("check_status", "generate_success")
    builder.add_edge("generate_success", END)
    
    # Compile graph
    graph = builder.compile()
    
    print("✅ Prompt chain graph compiled with validation node")
    return graph

# Create the graph
graph = create_prompt_chain_graph()

## 12. Visualize the Workflow

Display the graph structure showing the prompt chaining flow with conditional branches.

In [ ]:
# Visualize the graph structure
try:
    from IPython.display import Image, display
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception as e:
    print("Graph visualization:")
    print("""
    ┌─────────────────┐
    │     START       │
    └────────┬────────┘
             ↓
    ┌─────────────────┐
    │  extract_info   │  ← Step 1: Extract order ID from query
    └────────┬────────┘
             ↓
    ┌─────────────────┐
    │database_lookup  │  ← Step 2: Query database
    └────────┬────────┘
             ↓
    ┌─────────────────┐
    │   validation    │  ← Step 3: Validate & handle errors
    └────────┬────────┘
             ↓
         [Found?]
      ┌──────┴──────┐
     Yes            No
      ↓              ↓
    ┌────────┐    (END with error)
    │ check  │
    │ status │  ← Step 4: Get status info
    └───┬────┘
        ↓
    ┌────────┐
    │generate│
    │success │  ← Step 5: Create response
    └───┬────┘
        ↓
    ┌─────────────────┐
    │       END       │
    └─────────────────┘
    """)

## 13. Test the Prompt Chain

Run the complete workflow with different test queries to see the prompt chain in action.

In [ ]:
def run_order_enquiry(query: str):
    """Execute the prompt chain with a user query"""
    print("="*70)
    print(f"💬 USER QUERY: {query}")
    print("="*70)
    
    try:
        # Invoke the graph
        result = graph.invoke({
            "user_query": query,
            "extracted_info": None,
            "order_data": None,
            "customer_data": None,
            "order_found": False,
            "status_info": None,
            "final_response": ""
        })
        
        print("\n" + "="*70)
        print("🤖 ASSISTANT RESPONSE:")
        print("="*70)
        print(result["final_response"])
        print("\n")
        
        return result
        
    except Exception as e:
        print(f"\n❌ Error: {e}")
        return None

print("✅ Test function ready")

### Test Case 1: Successful Order Lookup (In Transit)

In [ ]:
run_order_enquiry("When will I get my order 101 delivered?")

### Test Case 2: Successful Order Lookup (Delivered)

In [ ]:
run_order_enquiry("Has my order #102 arrived yet?")

### Test Case 3: Order Not Found

In [ ]:
run_order_enquiry("Where is my order 999?")

### Test Case 4: Processing Order

In [ ]:
run_order_enquiry("What's the status of order 103?")

## 14. Interactive Testing Widget

Create an interactive interface to test the prompt chain with custom queries.

In [ ]:
try:
    import ipywidgets as widgets
    from IPython.display import display, clear_output
    
    # Create widgets
    query_input = widgets.Textarea(
        value='',
        placeholder='Enter your order enquiry here... (e.g., "Where is my order 101?")',
        description='Your Query:',
        layout=widgets.Layout(width='600px', height='80px'),
        style={'description_width': '100px'}
    )
    
    # Predefined test queries
    test_queries_dropdown = widgets.Dropdown(
        options=[
            ('-- Select a test query --', ''),
            ('Order 101 - In Transit', 'When will I get my order 101 delivered?'),
            ('Order 102 - Delivered', 'Has my order #102 arrived?'),
            ('Order 103 - Processing', "What's the status of order 103?"),
            ('Order Not Found', 'Where is my order 999?'),
            ('Casual Query', "Hi, I'm Sarah checking on order 101"),
        ],
        description='Examples:',
        layout=widgets.Layout(width='600px'),
        style={'description_width': '100px'}
    )
    
    run_button = widgets.Button(
        description='Run Query',
        button_style='primary',
        icon='play'
    )
    
    output_area = widgets.Output()
    
    def on_dropdown_change(change):
        if change['new']:
            query_input.value = change['new']
    
    def on_run_click(button):
        with output_area:
            clear_output()
            if query_input.value.strip():
                run_order_enquiry(query_input.value)
            else:
                print("⚠️  Please enter a query")
    
    test_queries_dropdown.observe(on_dropdown_change, names='value')
    run_button.on_click(on_run_click)
    
    # Display interface
    display(widgets.VBox([
        widgets.HTML("<h3>🧪 Interactive Prompt Chain Tester</h3>"),
        test_queries_dropdown,
        query_input,
        run_button,
        output_area
    ]))
    
except ImportError:
    print("⚠️  ipywidgets not available. Use run_order_enquiry() function instead.")

## 15. Key Learnings: Prompt Chaining Pattern

### ✅ **When to Use Prompt Chaining**
- Tasks with **predetermined steps** in a fixed sequence
- When you need **predictable, observable workflows**
- For **structured information extraction and transformation**
- When debugging and testing discrete steps is important

### ⚠️ **When NOT to Use Prompt Chaining**
- Tasks requiring **dynamic tool selection**
- Problems that need **adaptive decision-making**
- Scenarios where the solution path is **unpredictable**
- Complex multi-step reasoning that requires iteration

### 📊 **Pattern Comparison**

| Aspect | Prompt Chaining | Agentic Pattern |
|--------|----------------|------------------|
| **Flow** | Fixed sequence | Dynamic loops |
| **Decision Making** | Predetermined | Self-directed |
| **Predictability** | High | Variable |
| **Observability** | Excellent | Moderate |
| **Flexibility** | Low | High |
| **Use Case** | Structured tasks | Open-ended problems |

### 💡 **Best Practices**
1. **Store raw data in state** - Format in nodes as needed
2. **One responsibility per node** - Keep nodes focused and testable
3. **Clear state updates** - Each node should update specific state fields
4. **Graceful error handling** - Provide clear error paths and messages
5. **Detailed logging** - Track execution flow for debugging

### 🎯 **This Implementation Demonstrated**
- ✅ Sequential LLM calls with context passing
- ✅ Conditional branching based on intermediate results
- ✅ Database integration within a prompt chain
- ✅ Error handling with alternative paths
- ✅ Personalized response generation
- ✅ State management with typed dictionaries

## 16. Available Test Data Reference

### 📋 **Valid Order IDs for Testing**
Use any of these order IDs to test the workflow:
- **101-118** - Various orders with different statuses (in_transit, delivered, processing, cancelled)
- Try: 101, 102, 103, 105, 106, 109, 111, 114, 117

### 📝 **Sample Queries to Try**
```
"When will I get my order 101 delivered?"
"Has my order #102 arrived yet?"
"What's the status of order 103?"
"Where is my order 999?" (not found)
"I'm Sarah checking on order 101"
"Track my order 105 please"
"Order 106 status?" (cancelled order)
"What's happening with my order 114?"
```

### 📊 **Order Status Types**
- **in_transit**: Orders currently being shipped
- **delivered**: Successfully delivered orders
- **processing**: Orders being prepared
- **cancelled**: Cancelled orders

---

## 📚 Resources
- [LangGraph Documentation](https://langchain-ai.github.io/langgraph/)
- [Thinking in LangGraph](https://docs.langchain.com/oss/python/langgraph/thinking-in-langgraph)
- [Workflows vs Agents](https://docs.langchain.com/oss/python/langgraph/workflows-agents)

---

*This notebook demonstrates a production-ready prompt chaining workflow for automated customer service.*